<a href="https://colab.research.google.com/github/sahithikodali1/Summarization-of-Biomedical-evidence/blob/Data_processing/BIOASQ_Bert_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
import numpy as np
import pandas as pd
import os
import json
import csv
import pickle
import matplotlib.pyplot as plt
% matplotlib inline

# install
!pip install pytorch-pretrained-bert pytorch-nlp

# BERT imports
import torch

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange


In [33]:
##Code to load jsonfile

# from pandas.io.json import json_normalize 

# with open('/content/drive/MyDrive/Thesis_B/training9b.json', 'r') as json_file:
#     data = json.load(json_file)

In [34]:
##Code to get questions list

# data_access = data['questions']
# body_list = []
# for i in range(len(data_access)):
#   text_list = []
#   body_list.append(data_access[i]['body'])
#   for j in data_access[i]['snippets']:
#     text_list.append(j['text'])
#   # list_concat = body_list+text_list
#   # body_text_list.append(list_concat)


In [35]:
## Code to add questions to csv data

#DATA_DIR = "/content/drive/MyDrive/Thesis_B"
# file = pd.read_csv(os.path.join(DATA_DIR,'rouge_9b_mod.csv'))
# f_values = file.values

# qids = f_values[:,0]
# question_array = np.array(body_list)

# questions = []
# for i in range(len(qids)):
#   qid = qids[i]
#   questions.append(question_array[qid])

# file['question'] = questions
# file.to_csv('/content/drive/MyDrive/Thesis_B/9b_data.csv', index=False)

In [36]:
# #Code to split based on qid

# DATA_DIR = "/content/drive/MyDrive/Thesis_B"
# file = pd.read_csv(os.path.join(DATA_DIR,'9b_data.csv'))
# f_values = file.values
# q_id_split = np.split(f_values, np.where(np.diff(f_values[:,0]))[0]+1)
# print(q_id_split[:2])

[array([[0, '55031181e9bde69634000014', 20, 0.5814, 0.57143,
        0.5121899999999999, 0.44998999999999995, 0.52892,
        "The non-Mendelian inheritance of sporadic non-syndromic Hirschsprung's disease proved to be complex; involvement of multiple loci was demonstrated in a multiplicative model",
        1,
        'Is Hirschsprung disease a mendelian or a multifactorial disorder?'],
       [0, '55031181e9bde69634000014', 3, 0.5814, 0.57143,
        0.5121899999999999, 0.44998999999999995, 0.52892,
        "The non-Mendelian inheritance of sporadic non-syndromic Hirschsprung's disease proved to be complex; involvement of multiple loci was demonstrated in a multiplicative model",
        1,
        'Is Hirschsprung disease a mendelian or a multifactorial disorder?'],
       [0, '55031181e9bde69634000014', 16, 0.38961, 0.26666999999999996,
        0.21916999999999998, 0.16902, 0.25117,
        "The majority of the identified genes are related to Mendelian syndromic forms of Hirschsp

In [37]:
# #Predict train and test sizes

# train_data_size = round(len(q_id_split)*0.8)
# test_data_size = round(len(q_id_split) - train_data_size)
# print(train_data_size)
# print(test_data_size)

2994
749


In [38]:
# #Shuffle qid randomly

# import random
# random.seed(3007)
# random.shuffle(q_id_split)
# print(type(q_id_split[:1]))

<class 'list'>


In [39]:
# #Split train data into train and validation

# train_data_beforeval = q_id_split[:2994]
# f_values_train_beforeval = np.concatenate(train_data_beforeval, axis=0)
# # print((f_values_train_beforeval))

# q_id_split_train = np.split(f_values_train_beforeval, np.where(np.diff(f_values_train_beforeval[:,0]))[0]+1)
# val_data_size = round(len(q_id_split_train)*0.2)
# train_data_size = round(len(q_id_split_train) - val_data_size)
# print(train_data_size)
# print(val_data_size)

2395
599


In [40]:
# #Split train, val, test data

# train_data = q_id_split_train[:2395]
# val_data = q_id_split_train[2395:]
# test_data = q_id_split[2994:]
# f_values_train = np.concatenate(train_data, axis=0)
# f_values_val = np.concatenate(val_data, axis=0)
# f_values_test = np.concatenate(test_data, axis=0)
# col_names = file.columns
# print(col_names)

Index(['qid', 'pubmedid', 'sentid', 'N1', 'N2', 'L', 'S4', 'SU4',
       'sentence text', 'SU4_labels', 'question'],
      dtype='object')


In [41]:
# #Convert train,val,test data into dataframes

# train_df = pd.DataFrame(f_values_train, columns = col_names)
# val_df = pd.DataFrame(f_values_val, columns = col_names)
# test_df = pd.DataFrame(f_values_test, columns = col_names)

In [42]:
# print(len(train_data))
# print(len(val_data))
# print(len(test_data))

# print(len(train_df))
# print(len(val_df))
# print(len(test_df))
# print(train_df.columns.values)

2395
599
749
37178
8494
11219
['qid' 'pubmedid' 'sentid' 'N1' 'N2' 'L' 'S4' 'SU4' 'sentence text'
 'SU4_labels' 'question']


In [43]:
# def obtain_specialtokenized_list(dataframe):
#   sentences = dataframe['sentence text']
#   questions = dataframe['question']
#   sentences_list = list(sentences)
#   questions_list = list(questions)
#   question_sentence_list = []
#   for i in range(len(sentences_list)):
#     question_sentence_list = question_sentence_list + ["[CLS] " + questions_list[i] + " [SEP] " + sentences_list[i] + " [SEP]"]
#   return question_sentence_list

In [44]:
# train_specialtok_list = obtain_specialtokenized_list(train_df)  
# val_specialtok_list = obtain_specialtokenized_list(val_df)  

In [45]:
# def obtain_SU4labels_list(dataframe):
#   SU4_labels = dataframe['SU4_labels']
#   labels_list = list(SU4_labels)
#   return labels_list


# train_labels = obtain_SU4labels_list(train_df)
# val_labels = obtain_SU4labels_list(val_df)

In [52]:
# #Write labels

# with open("/content/drive/MyDrive/Thesis_B/labels_train.txt", "wb") as fp:   #Pickling
#   pickle.dump(train_labels, fp)

# with open("/content/drive/MyDrive/Thesis_B/labels_val.txt", "wb") as fp:   #Pickling
#   pickle.dump(val_labels, fp)

In [17]:
##Write n Read Specialtok sentences

# import pickle
# with open("/content/drive/MyDrive/Thesis_B/Specialtok_sent_train.txt", "wb") as fp:   #Pickling
#   pickle.dump(train_specialtok_list, fp)

# with open("/content/drive/MyDrive/Thesis_B/Specialtok_sent_val.txt", "wb") as fp:   #Pickling
#   pickle.dump(val_specialtok_list, fp)

# with open("/content/drive/MyDrive/Thesis_B/Specialtok_sent_train.txt", "rb") as fp:   # Unpickling
#   train_specialtok_list = pickle.load(fp)

# with open("/content/drive/MyDrive/Thesis_B/Specialtok_sent_val.txt", "rb") as fp:   # Unpickling
#   val_specialtok_list = pickle.load(fp)


In [18]:
# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla K80'

In [19]:
# Tokenize with BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


100%|██████████| 231508/231508 [00:00<00:00, 698807.68B/s]


In [20]:
# def tokenize_sent(givenlist):
#   tokenized_texts = [tokenizer.tokenize(sent) for sent in givenlist]
#   return tokenized_texts

In [21]:
# train_tokenized = tokenize_sent(train_specialtok_list)
# val_tokenized = tokenize_sent(val_specialtok_list)

In [22]:
# print(type(train_tokenized))
# print(len(train_tokenized))
# print(train_tokenized[0])

In [23]:
##Write tokenized sentences

# with open("/content/drive/MyDrive/Thesis_B/Tokenized_sent_train.txt", "wb") as fp:   #Pickling
#   pickle.dump(train_tokenized, fp)

# with open("/content/drive/MyDrive/Thesis_B/Tokenized_sent_val.txt", "wb") as fp:   #Pickling
#   pickle.dump(val_tokenized, fp)

In [54]:
#Read tokenized sentences

with open("/content/drive/MyDrive/Thesis_B/Tokenized_sent_train.txt", "rb") as fp:   # Unpickling
  train_tokenized = pickle.load(fp)

with open("/content/drive/MyDrive/Thesis_B/Tokenized_sent_val.txt", "rb") as fp:   # Unpickling
  val_tokenized = pickle.load(fp)

In [55]:
#Read labels

with open("/content/drive/MyDrive/Thesis_B/labels_train.txt", "rb") as fp:   # Unpickling
  train_labels = pickle.load(fp)

with open("/content/drive/MyDrive/Thesis_B/labels_val.txt", "rb") as fp:   # Unpickling
  val_labels = pickle.load(fp)

In [56]:
print(type(train_tokenized))
print(len(train_tokenized))
print(train_tokenized[0])
print(len(val_tokenized))
print(val_tokenized[0])

<class 'list'>
37178
['[CLS]', 'what', 'is', 'mas', '##iti', '##ni', '##b', 'an', 'inhibitor', 'of', '?', '[SEP]', 'this', 'study', 'evaluated', 'the', 'therapeutic', 'potential', 'of', 'mas', '##iti', '##ni', '##b', ',', 'an', 'oral', 'ty', '##ros', '##ine', 'kinase', 'inhibitor', 'with', 'activity', 'against', 'c', '-', 'kit', 'and', 'plate', '##let', '-', 'derived', 'growth', 'factor', 'receptors', '(', 'pd', '##gf', '##r', ')', ',', 'to', 'reduce', 'is', '##che', '##mic', 'brain', 'area', 'and', 'neurological', 'deficit', '.', '[SEP]']
8494
['[CLS]', 'which', 'proteins', 'are', 'regulated', 'by', 'nr', '##f', '##2', '?', '[SEP]', 'ke', '##ap', '##1', '-', 'nr', '##f', '##2', 'system', 'is', 'known', 'as', 'a', 'sensor', 'of', 'electro', '##phi', '##lic', 'compounds', ',', 'and', 'protects', 'cells', 'from', 'ox', '##ida', '##tive', 'stress', 'through', 'induction', 'of', 'various', 'anti', '##ox', '##ida', '##nt', 'enzymes', '.', '[SEP]']


In [57]:
#Convert tokenized sentences to respective token ids

def token2ids(tokenized_texts):
  # Set the maximum sequence length. 
  MAX_LEN = 512

  tokens_to_ids = [tokenizer.convert_tokens_to_ids(sent) for sent in tokenized_texts]
  tokens_to_ids = pad_sequences(tokens_to_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

  return tokens_to_ids

In [58]:
train_tokenids = token2ids(train_tokenized)
val_tokenids = token2ids(val_tokenized)

In [59]:
#Create masks for the tokenids

def create_masks(token_ids):
  attention_masks = []

  for tid in token_ids:
    tid_mask = [float(i>0) for i in tid]
    attention_masks.append(tid_mask)
  return attention_masks

In [60]:
train_masks = create_masks(train_tokenids)
val_masks = create_masks(val_tokenids)

In [61]:
print(len(train_tokenids))
print(len(train_masks))
print(train_tokenids[0])
print(train_masks[0])

37178
37178
[  101  2054  2003 16137 25090  3490  2497  2019 24054  1997  1029   102
  2023  2817 16330  1996 17261  4022  1997 16137 25090  3490  2497  1010
  2019  8700  5939  7352  3170 21903 24054  2007  4023  2114  1039  1011
  8934  1998  5127  7485  1011  5173  3930  5387 13833  1006 22851 25708
  2099  1007  1010  2000  5547  2003  5403  7712  4167  2181  1998 23130
 15074  1012   102     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0   

In [63]:
print(type(train_tokenids))
print(type(train_masks))
print(type(train_labels))

print(train_tokenids.shape)
print(len(train_masks))
print(len(train_labels))

<class 'numpy.ndarray'>
<class 'list'>
<class 'list'>
(37178, 512)
37178
37178


In [64]:
# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_tokenids, dtype=torch.int64)
validation_inputs = torch.tensor(val_tokenids, dtype=torch.int64)

train_labels = torch.tensor(train_labels, dtype=torch.int64)
validation_labels = torch.tensor(val_labels, dtype=torch.int64)

train_masks = torch.tensor(train_masks, dtype=torch.float32)
validation_masks = torch.tensor(val_masks, dtype=torch.float32)

In [65]:
print(type(train_inputs))
print(type(train_labels))
print(type(train_masks))

print(train_inputs.dtype)
print(train_labels.dtype)
print(train_masks.dtype)

print(validation_inputs.dtype)
print(validation_labels.dtype)
print(validation_masks.dtype)

print(train_inputs.shape)
print(train_labels.shape)
print(train_masks.shape)

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
torch.int64
torch.int64
torch.float32
torch.int64
torch.int64
torch.float32
torch.Size([37178, 512])
torch.Size([37178])
torch.Size([37178, 512])


In [66]:
!pip install transformers
from transformers import get_linear_schedule_with_warmup

     |████████████████████████████████| 2.1MB 10.1MB/s 
     |████████████████████████████████| 3.3MB 40.8MB/s 
     |████████████████████████████████| 901kB 38.1MB/s 


In [67]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top.

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 2)

for param in model.bert.parameters():
    param.requires_grad = False
    
model.cuda()

100%|██████████| 407873900/407873900 [00:13<00:00, 29776450.07B/s]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [68]:
#BERT fine-tuning parameters
lr=2e-5
num_warmup_steps = 10
num_training_steps = 1000

optimizer = torch.optim.AdamW(model.parameters(), lr = lr)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = num_warmup_steps, num_training_steps = num_training_steps)

In [70]:
# Select a batch size for training. 
batch_size = 32

# Create an iterator of our data with torch DataLoader 
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data) #change to seq sampler next time and try
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
# for step, batch in enumerate(train_dataloader):
#   print(step)
#   b_input_ids, b_input_mask, b_labels = batch
#   print(b_input_ids.size(0))
#   # print(len(b_input_mask))
#   # print(len(b_labels))

In [71]:
import random
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
# train_loss_set = []
# # train_logits_set = []
# epochs = 3

# for _ in trange(epochs, desc="Epoch"): 
#   model.train() 
#   tr_loss = 0
#   nb_tr_examples, nb_tr_steps = 0, 0

#   for step, batch in enumerate(train_dataloader):

#     print("Step:{}".format(step))

#     batch = tuple(t.to(device) for t in batch)
#     b_input_ids, b_input_masks, b_labels = batch

#     optimizer.zero_grad()

#     loss = model(b_input_ids, token_type_ids = None, attention_mask = b_input_masks, labels = b_labels)
#     print("Loss for batch :{}".format(loss))
#     # print("Logits for batch :{}".format(logits))
#     train_loss_set.append(loss.item()) 
#     # train_logits_set.append(logits.item()) 

#     loss.backward()
#     optimizer.step()
#     scheduler.step()

#     tr_loss = tr_loss + loss.item()
#     print("tr_loss:{}".format(tr_loss))

#     nb_tr_examples += b_input_ids.size(0)
#     print("nb_tr_examples:{}".format(nb_tr_examples))

#     nb_tr_steps += 1
#     print("nb_tr_steps:{}".format(nb_tr_steps))

#   print(" Train loss: {}".format(tr_loss/nb_tr_steps))

In [72]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs 
epochs = 3

for _ in trange(epochs, desc="Epoch"):  
  ## TRAINING
  # Set our model to training mode
  model.train()  
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):

    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_masks, b_labels = batch

    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    
    # Forward pass
    loss = model(b_input_ids, token_type_ids = None, attention_mask = b_input_masks, labels = b_labels)
    train_loss_set.append(loss.item())  
    # train_logits_set.append(logits.item())   
    

    # Backward pass
    loss.backward()
    
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    scheduler.step()
    
    # Update tracking variables
    tr_loss += loss.item() #(Transfers to a plain python number as that only can live on CPU)
    
    nb_tr_examples += b_input_ids.size(0)
    
    nb_tr_steps += 1

  print(" Train loss: {}".format(tr_loss/nb_tr_steps))

Epoch:  33%|███▎      | 1/3 [44:52<1:29:45, 2692.92s/it]

 Train loss: 0.615616526461099


Epoch:  67%|██████▋   | 2/3 [1:29:34<44:49, 2689.43s/it]

 Train loss: 0.608901037227061


Epoch: 100%|██████████| 3/3 [2:14:16<00:00, 2685.54s/it]

 Train loss: 0.6088739113943097


In [73]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [74]:
# VALIDATION

logits_list = []
# Put model in evaluation mode
model.eval()

# Tracking variables 
eval_loss, eval_accuracy, eval_f1score = 0, 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# Evaluate data for one epoch
for batch in validation_dataloader:

  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_masks, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and speeding up validation
  with torch.no_grad():
    
    # Forward pass, calculate logit predictions
    logits = model(b_input_ids, token_type_ids = None, attention_mask = b_input_masks)   
    print("Logits_each:{}".format(logits))
    logits_list.append(logits)
  
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  print("Logits:{}".format(logits))
  print("Labels:{}".format(label_ids))
  
  tmp_eval_accuracy = flat_accuracy(logits, label_ids)
  # tmp_eval_f1score = flat_f1score(logits, label_ids)    
  eval_accuracy += tmp_eval_accuracy
  # eval_f1score += tmp_eval_f1score
  nb_eval_steps += 1

print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
# print("Validation f1score: {}".format(eval_f1score/nb_eval_steps))


Streaming output truncated to the last 5000 lines.
 [ 0.3096775  -0.52360564]
 [ 0.34029275 -0.43659687]
 [ 0.37122482 -0.5126048 ]
 [ 0.40293613 -0.43832722]
 [ 0.35754216 -0.4249533 ]
 [ 0.49490553 -0.45117253]
 [ 0.40284264 -0.43330753]
 [ 0.4922052  -0.4499023 ]
 [ 0.4922052  -0.4499023 ]
 [ 0.4000904  -0.42449948]
 [ 0.43301797 -0.4456572 ]
 [ 0.38505372 -0.42947805]
 [ 0.49580994 -0.51767844]
 [ 0.49580994 -0.51767844]
 [ 0.49580994 -0.51767844]
 [ 0.5020286  -0.47296157]
 [ 0.5020286  -0.47296157]
 [ 0.40437597 -0.46629772]
 [ 0.43338394 -0.47650668]
 [ 0.3854981  -0.49821627]
 [ 0.3854981  -0.49821627]
 [ 0.38791537 -0.45711148]
 [ 0.45707613 -0.49449468]
 [ 0.45748755 -0.51141864]
 [ 0.33420643 -0.53976583]
 [ 0.33420643 -0.53976583]
 [ 0.33420643 -0.53976583]
 [ 0.4597324  -0.5145579 ]
 [ 0.37594172 -0.48977098]]
Labels:[0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Logits_each:tensor([[ 0.4549, -0.4877],
        [ 0.3776, -0.5072],
        [ 0.4946, -0.429

In [ ]:
# torch.save(model.state_dict(), '/content/drive/MyDrive/Thesis_B/weights_default')

In [ ]:
# model.load_state_dict(torch.load('/content/drive/MyDrive/Thesis_B/weights_default'))
# model.eval()

In [ ]:
# # load the model checkpoint
# checkpoint = torch.load('../outputs/model.pth')
# # load model weights state_dict
# model.load_state_dict(checkpoint['model_state_dict'])
# print('Previously trained model weights state_dict loaded...')
# # load trained optimizer state_dict
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# print('Previously trained optimizer state_dict loaded...')
# epochs = checkpoint['epoch']
# # load the criterion
# criterion = checkpoint['loss']
# print('Trained model loss function loaded...')
# print(f"Previously trained for {epochs} number of epochs...")
# # train for more epochs
# epochs = new_epochs
# print(f"Train for {epochs} more epochs...")

In [ ]:
# # save model checkpoint
# torch.save({
#             'epoch': epochs,
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             'loss': criterion,
#             }, '/content/drive/MyDrive/Thesis_B/outputs/model.pth')